# Race to low rms

Import latin-hypercube test set

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import utils_intensity_map as uim
import utils_deck_generation as idg
import netcdf_read_write as nrw
import training_data_generation as tdg
import tf_neural_network as tfnn
import time
import os
import shutil
import utils_optimizers as uopt
import copy
%matplotlib inline
plt.ion()
run_dir = "Data_input"
trainingdata_filename = "flipped_training_data_and_labels.nc"

In [ ]:
sys_params = tdg.define_system_params(run_dir)
sys_params["trainingdata_filename"] = trainingdata_filename
X_all, Y_all, avg_powers_all = nrw.import_training_data(sys_params)

In [ ]:
target_all = np.sqrt(np.sum(Y_all**2, axis=0))

target_mean = np.mean(target_all)
target_variance = np.sqrt(np.var(target_all))
print(target_mean*100.0, target_variance*100.0)

input_means = np.mean(X_all, axis=1)
input_standard_deviation = np.sqrt(np.var(X_all, axis=1))
print(input_means*100.0, input_standard_deviation*100.0)

In [ ]:
print(np.shape(X_all[:,0]), np.shape(Y_all[:,0]))
mindex = np.argmin(np.mean(Y_all, axis=0))
print(mindex)
print(np.sum(Y_all[:,mindex]))
print(target_all[mindex])
mindex = np.argmin(target_all)
print(mindex)
print(np.sum(Y_all[:,mindex]))
print(target_all[mindex])

num_init_examples = np.shape(X_all)[1]
print(num_init_examples)
num_inputs = np.shape(X_all)[0]
num_modes = np.shape(Y_all)[0]
print(num_inputs, num_modes)

In [ ]:
fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(num_modes), Y_all[:,mindex] * 100.0)
ax.set_xticks(range(0, num_modes+1, int(num_modes/5)))
plt.xlim([0, num_modes])
plt.title("Unweighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"amplitude ($\%$)");

## Method 1, Brute force

In [ ]:
##

## Method 2, Gradient descent

The partial derivative is determined using a 2*16=32 grid of points (2 points in every dimension) around the current minima. These points can be evaluated in either a NN or Ifriit depending on speed.

In [ ]:
root_dir = "Data_output"
num_parallel = 11
n_iter = 10
optimizer_params = uopt.define_optimizer_parameters(root_dir, num_inputs, num_modes, num_init_examples, n_iter, num_parallel)
dataset_optimizer = uopt.define_optimizer_dataset(X_all, Y_all, avg_powers_all)
grad_descent_params = uopt.define_gradient_descent_params(num_parallel)
grad_descent_params = uopt.wrapper_gradient_descent(dataset_optimizer, grad_descent_params, optimizer_params)

## Method 3, Use surrogate NN to pick low RMS from random inputs

## Method 4, Use inverse NN to indentify low rms by inputing other low rms cases

## Method 5, Genetic algorithm

Iterative procedure taking best features of first generation. Mutate and mix inputs between the best and produce subsequent generation.

In [ ]:
init_points = 10
n_iter = 10
num_parallel = 10
run_dir = "Data_empty"
iter_dir = "iter_"
filename_flipped_trainingdata = "flipped_training_data_and_labels.nc"

In [ ]:
# Defining the population size.
pop_size = (num_inputs, init_points) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.
#Creating the initial population.
X_pop = rng.random(pop_size)
num_parents_mating = int(init_points / 10.0)
if (num_parents_mating % 2) != 0:
    num_parents_mating -=1

pbounds = np.zeros((num_inputs, 2))
pbounds[:,1] = 1.0

best_outputs = []
X_all = np.array([], dtype=np.int64).reshape(num_inputs,0)
Y_all= np.array([], dtype=np.int64).reshape(LMAX,0)
avg_powers_all = np.array([], dtype=np.int64)
tic = time.perf_counter()
for generation in range(n_iter-1):
    print("Generation : ", generation)
    # Measuring the fitness of each chromosome in the population.
    Y_pop, avg_powers_pop = tdg.run_ifriit_input(init_points, X_pop, run_dir, LMAX, num_parallel, hemisphere_symmetric, run_clean)
    X_all = np.hstack((X_all, X_pop))
    Y_all = np.hstack((Y_all, Y_pop))
    avg_powers_all = np.hstack((avg_powers_all, avg_powers_pop))
    for irun in range(init_points):
        os.rename(run_dir + "/run_" + str(irun), run_dir + "/" + iter_dir + str(irun+init_points*generation))
        
    fitness_pop = -np.sqrt(np.sum(Y_pop**2, axis=0))
    mindex_pop = np.argmax(fitness_pop)
    
    if (generation+1)%10 <= 0.0:
        toc = time.perf_counter()
        print("{:0.4f} seconds".format(toc - tic))
        print(str(generation+1) + " genetic algorithm data points added, saving to .nc")
        filename_trainingdata = run_dir + '/' + filename_flipped_trainingdata
        nrw.save_training_data(X_all, Y_all, avg_powers_all, filename_trainingdata)
        mindex = np.argmin(np.mean(Y_all, axis=0))
        print(mindex)
        print(np.sum(Y_all[:,mindex]))
        print(np.sqrt(np.sum(Y_all[:,mindex]**2)))
        mindex = np.argmin(np.sqrt(np.sum(Y_all**2, axis=0)))
        print(mindex)
        print(np.sum(Y_all[:,mindex]))
        print(np.sqrt(np.sum(Y_all[:,mindex]**2)))

    best_outputs.append(np.max(fitness_pop))
    # The best result in the current iteration.
    print("Best result : ", best_outputs[generation])

    # Selecting the best parents in the population for mating.
    parents = uopt.select_mating_pool(X_pop.T, fitness_pop, num_parents_mating)

    # Generating next generation using crossover.
    offspring_crossover = uopt.crossover(parents, offspring_size=(init_points-num_parents_mating, num_inputs))

    # Adding some variations to the offspring using mutation.
    offspring_mutation = uopt.mutation(offspring_crossover, rng, pbounds, num_mutations=2)

    # Creating the new population based on the parents and offspring.
    X_pop[:,0:num_parents_mating] = parents.T
    X_pop[:,num_parents_mating:] = offspring_mutation.T
# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
Y_pop, avg_powers_pop = tdg.run_ifriit_input(init_points, X_pop, run_dir, LMAX, num_parallel, hemisphere_symmetric, run_clean)
X_all = np.hstack((X_all, X_pop))
Y_all = np.hstack((Y_all, Y_pop))
avg_powers_all = np.hstack((avg_powers_all, avg_powers_pop))
for irun in range(init_points):
    os.rename(run_dir + "/run_" + str(irun), run_dir + "/" + iter_dir + str(irun+init_points*(generation+1)))

# Then return the index of that solution corresponding to the best fitness.
fitness_pop = -np.sqrt(np.sum(Y_pop**2, axis=0))
mindex_pop = np.argmax(fitness_pop)

print("Best solution : ", X_pop[:, mindex_pop])
print("Best solution fitness : ", fitness_pop[mindex_pop])

toc = time.perf_counter()
print("{:0.4f} seconds".format(toc - tic))
print(str(generation+1) + " genetic algorithm data points added, saving to .nc")
filename_trainingdata = run_dir + '/' + filename_flipped_trainingdata
nrw.save_training_data(X_all, Y_all, avg_powers_all, filename_trainingdata)
mindex = np.argmin(np.mean(Y_all, axis=0))
print(mindex)
print(np.sum(Y_all[:,mindex]))
print(np.sqrt(np.sum(Y_all[:,mindex]**2)))
mindex = np.argmin(np.sqrt(np.sum(Y_all**2, axis=0)))
print(mindex)
print(np.sum(Y_all[:,mindex]))
print(np.sqrt(np.sum(Y_all[:,mindex]**2)))

#plt.plot(best_outputs)
#plt.xlabel("Iteration")
#plt.ylabel("Fitness")
#plt.show()

## Method 6, Bayesian optimization

Gaussian process surrogate and bayesian optimization used with multiple sources of information. First we create a bayesian model with the true data points and select new simulations based on that. The model (Kriging method?) could use a "gaussian process approximation" to reduce computational expense.

Future extension: multi-source bayesian optimization, (ifriit is high quality source and NN is low quality)

In [ ]:
init_points = num_init_examples
n_iter = 10
run_dir = "Data_output"
iter_dir = "iter_"
num_parallel = 10 # Not currently run parallel

In [ ]:
pbounds = {}
for ii in range(num_inputs):
    pbounds["x"+str(ii)] = (0., 1.)

target = -np.sqrt(np.sum(Y_all**2, axis=0)) # Critical to make negative (min not max)
print(num_inputs, init_points, np.shape(target))

optimizer, utility = uopt.initialize_unknown_func(X_all, target, pbounds, init_points, num_inputs)
print(optimizer.max)

In [ ]:
tic = time.perf_counter()
for ieval in range(n_iter):
    next_point = optimizer.suggest(utility)

    X_new = np.zeros((num_inputs, 1))
    for ii in range(num_inputs):
        X_new[ii] = next_point["x"+str(ii)]
    
    Y_new, avg_powers_new = tdg.run_ifriit_input(1, X_new, run_dir, LMAX, num_parallel, hemisphere_symmetric, run_clean)
    
    X_all = np.hstack((X_all, X_new))
    Y_all = np.hstack((Y_all, Y_new))
    avg_powers_all = np.hstack((avg_powers_all, avg_powers_new))
    
    os.rename(run_dir + "/run_0", run_dir + "/" + iter_dir + str(ieval+num_init_examples))
    
    target = -np.sqrt(np.sum(Y_new**2))
    try:
        optimizer.register(params=next_point, target=target)
    except:
        print("Broken input!", next_point, target)
    if (ieval+1)%10 <= 0.0:
        toc = time.perf_counter()
        print("{:0.4f} seconds".format(toc - tic))
        print(str(ieval+1) + " Bayesian data points added, saving to .nc")
        filename_trainingdata = run_dir + '/' + filename_flipped_trainingdata
        nrw.save_training_data(X_all, Y_all, avg_powers_all, filename_trainingdata)
        print(optimizer.max)
        mindex = np.argmin(np.mean(Y_all, axis=0))
        print(mindex)
        print(np.sum(Y_all[:,mindex]))
        print(np.sqrt(np.sum(Y_all[:,mindex]**2)))
        mindex = np.argmin(np.sqrt(np.sum(Y_all**2, axis=0)))
        print(mindex)
        print(np.sum(Y_all[:,mindex]))
        print(np.sqrt(np.sum(Y_all[:,mindex]**2)))
print(next_point)

## Method 7, Grid search algorithm

Split the entire search space into a grid (start coarse 2 or 3 cells per dimension) 3^16 = 43M. Evaluate each cell depending on the data points within or 8 nearest neighbours.

## Method 8, Network search algorithm

Find a gradient between all data points, use this information to initialize gradient descent

## Method 9, Principle Component Analysis (PCA)

Combine with gradient descent for faster convergence? Enables plotting of dataset in 2D

## Method 10, Transfer Learning 

Generate low quality large dataset (1-50M examples?) using surrogate NN and use this for transfer learning. This might help to evaluate at what stage transfer learning becomes effective (can we use it with a dataset of 1000 or 10000?)

## Method 11, Parallelised Bayesian optimization

In [ ]:
init_points = num_init_examples
n_iter = 10
num_parallel = 10
target_set_undetermined = - target_mean / 2.0
run_dir = "Data_221123b_ga_1kex_pbo"
iter_dir = "iter_"
filename_flipped_trainingdata = "flipped_training_data_and_labels.nc"

In [ ]:
pbounds = {}
for ii in range(num_inputs):
    pbounds["x"+str(ii)] = (0., 1.)

target = -np.sqrt(np.sum(Y_all**2, axis=0)) # Critical to make negative (min not max)
print(num_inputs, init_points, np.shape(target))

optimizer, utility = uopt.initialize_unknown_func(X_all, target, pbounds, init_points, num_inputs)
print(optimizer.max)

In [ ]:
tic = time.perf_counter()
for niter in range(n_iter):
    optimizer2 = copy.deepcopy(optimizer)
    
    X_new = np.zeros((num_inputs, num_parallel))
    for npar in range(num_parallel):
        next_point = optimizer2.suggest(utility)
        for ii in range(num_inputs):
            X_new[ii,npar] = next_point["x"+str(ii)]

        try:
            optimizer2.register(params=next_point, target=target_set_undetermined)
        except:
            print("Broken input!", next_point, target_set_undetermined)
    
    Y_new, avg_powers_new = tdg.run_ifriit_input(num_parallel, X_new, run_dir, LMAX, num_parallel, hemisphere_symmetric, run_clean)
    X_all = np.hstack((X_all, X_new))
    Y_all = np.hstack((Y_all, Y_new))
    avg_powers_all = np.hstack((avg_powers_all, avg_powers_new))
    
    for npar in range(num_parallel):
        ieval = niter * num_parallel + npar
        os.rename(run_dir + "/run_" + str(npar), run_dir + "/" + iter_dir + str(ieval+num_init_examples))

        target = -np.sqrt(np.sum(Y_new[:,npar]**2))
        for ii in range(num_inputs):
            next_point["x"+str(ii)] = X_new[ii,npar] 
        try:
            optimizer.register(params=next_point, target=target)
        except:
            print("Broken input!", next_point, target)

    if (niter+1)%10 <= 0.0:
        toc = time.perf_counter()
        print("{:0.4f} seconds".format(toc - tic))
        print(str(ieval+1) + " Bayesian data points added, saving to .nc")
        filename_trainingdata = run_dir + '/' + filename_flipped_trainingdata
        nrw.save_training_data(X_all, Y_all, avg_powers_all, filename_trainingdata)
        print(optimizer.max)
        mindex = np.argmin(np.mean(Y_all, axis=0))
        print(mindex)
        print(np.sum(Y_all[:,mindex]))
        print(np.sqrt(np.sum(Y_all[:,mindex]**2)))
        mindex = np.argmin(np.sqrt(np.sum(Y_all**2, axis=0)))
        print(mindex)
        print(np.sum(Y_all[:,mindex]))
        print(np.sqrt(np.sum(Y_all[:,mindex]**2)))
print(next_point)